In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sys
import math
from math import cos,atan,floor 
import numpy as np
import os
from sklearn import mixture
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import silhouette_score

In [2]:
def height_and_width(img):
    
    height = len(img)
    width = len(img[0])
    
    return height,width

def dimention_mesure_of(height,width):
    
    dimention_mesure=min(height,width)
    
    return dimention_mesure

def show_image_in_shape(width,height,img):
    cv2.namedWindow('',cv2.WINDOW_NORMAL)
    cv2.imshow('', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def angle_weight(angle):
    return math.fabs( cos( 2 * ( angle * (( math.pi * 2)/360) ) +(math.pi/2)  ) )


In [3]:
# Prepared kernels 

# empty 5 diamon :
#kernel = np.array([[0,0,1,0,0],[0,1,0,1,0],[1,0,0,0,1],[0,1,0,1,0],[0,0,1,0,0]],np.uint8) 
# vertical 5 line :
#kernel = np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0]],np.uint8)
# horizontal 5 line :
#kernel = np.array([[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0],[0,0,1,0,0]],np.uint8).transpose() 
# vertical 3 line :
#kernel = np.array([[0,1,0],[0,1,0],[0,1,0]],np.uint8)
# diag 3Pi/4 5 line :
#kernel = np.array([[1,0,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,0,0,1]],np.uint8) 
# diag Pi/4 5 line :
#kernel = np.array([[0,0,0,0,1],[0,0,0,1,0],[0,0,1,0,0],[0,1,0,0,0],[1,0,0,0,0]],np.uint8)

In [4]:

def kernel_of_angle(L,angle):
    M = np.zeros((L, L),np.uint8)
    for s in range(0,5*L):
        i = math.ceil(s*math.cos((angle * math.pi * 2)/360))
        j = math.ceil(s*math.sin((angle * math.pi * 2)/360))
        if (1-(L/2))<i and i<(L/2) and (1-(L/2))< j and j < (L/2) :
            M[ int(j+L/2),int(i+L/2) ]=1
            M[ int(-j+L/2), int(-i+L/2) ]=1
    return M

def kernel_of_radius(L):
    M = np.zeros((L, L),np.uint8)
    for s in range(0,2*360):
        i = math.ceil((-1+(L/2))*math.cos((s*math.pi/360)))
        j = math.ceil((-1+(L/2))*math.sin((s*math.pi/360)))
        if (1-(L/2))<i and i<(L/2) and (1-(L/2))< j and j < (L/2) :
            for k in range(0,L):
                M[ int(((k/L)*j+(L/2))),int(((k/L)*i+(L/2))) ]=1
    return M


In [5]:
def filtered_image(img,dimention_mesure,L,angle):

    img =  cv2.bitwise_not(img)
    img = cv2.GaussianBlur(img, (1, 1), 1)
    
    kernel = kernel_of_angle(L,angle)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel,iterations=1)
    
    labelnum, labelimg, contours, GoCs= cv2.connectedComponentsWithStats(img,cv2.CC_STAT_AREA  )
    
    surfaces_of_components=[]
    firstmaxs=[]
    secondmaxs=[]
    
    for label in range(1,labelnum):
        x,y = GoCs[label]
        img = cv2.circle(img, (int(x),int(y)), 1, (0,0,255), -1)
        x,y,w,h,size = contours[label]
        img = cv2.rectangle(img, (x,y), (x+w,y+h), (255,255,0), 1)
        surfaces_of_components.append(min(w,h))
    if surfaces_of_components:
        firstmax = max(surfaces_of_components)*1.0/dimention_mesure
        firstmaxs.append(firstmax)
        surfaces_of_components.remove(np.array(max(surfaces_of_components)))
        if surfaces_of_components:
            secondmax = max(surfaces_of_components)/dimention_mesure
        else:
            secondmax = 0
            secondmaxs.append(secondmax)
    else:
        firstmax = 0
        secondmax = 0
        
    #show_image_in_shape(600,600,img)
    
    return firstmax,secondmax


In [6]:
# test kernels
print(kernel_of_angle(30,150))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0

In [7]:
rootdir = '/media/cuong-nguyen/B0EA64A4EA646894/backup_ubuntu/2018/Projet/rentre_signee/Sourcecode/rentree_signee_v1.0/py-faster-rcnn/lib/process/detect_signature/Signature/signature_square/signature_yes'
k=0
nombre_limit = 10
min_score = 1000
max_score = 0
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if k < nombre_limit :

            #file = 'cut10.PNG'
            img = cv2.imread(os.path.join(subdir, file),0)
            cv2.imshow('', img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            height,width = height_and_width(img)
            dimention_mesure = dimention_mesure_of(height,width)
            L = 30
            #angle = 80

            signature_present = False
            maxfirstmax = 0
            for angle in range(0,180,5):
                firstmax,secondmax = filtered_image(img,dimention_mesure,L,angle)
                oblique_firstmax =  (angle_weight(angle)**(1/2)) * firstmax#(angle_weight(angle)**(0)) 
                if oblique_firstmax > 0.07 :
                    signature_present = True
                    ecart = oblique_firstmax - 0.04
                    #print(ecart)
                    #break;
                if oblique_firstmax > maxfirstmax :
                    maxfirstmax = oblique_firstmax
           
            if signature_present:
                print('signature in : ',file , ' with score : ' , maxfirstmax)
                print()
            else :
                print('No signature in : ',file , ' with score : ' , maxfirstmax)
                print()
            if maxfirstmax < min_score :
                min_score  = maxfirstmax
            if maxfirstmax > max_score:
                max_score  = maxfirstmax
            k=k+1

#print('Max score is : ' , max_score)
print('Min score is : ' , min_score)

('signature in : ', 'cut1.PNG', ' with score : ', 0.35340314136125656)
()
('signature in : ', 'cut10.PNG', ' with score : ', 0.076923076923076927)
()
('signature in : ', 'cut11.PNG', ' with score : ', 0.22781774580335731)
()
('signature in : ', 'cut12.PNG', ' with score : ', 0.15358361774744028)
()
('signature in : ', 'cut14.PNG', ' with score : ', 0.095744680851063829)
()
('signature in : ', 'cut15.PNG', ' with score : ', 0.1484848484848485)
()
('signature in : ', 'cut16.PNG', ' with score : ', 0.13625866050808313)
()
('signature in : ', 'cut18.PNG', ' with score : ', 0.25793650793650796)
()
('signature in : ', 'cut19.PNG', ' with score : ', 0.12572533849129594)
()
('signature in : ', 'cut2.PNG', ' with score : ', 0.15916955017301038)
()
('Min score is : ', 0.076923076923076927)


In [8]:

rootdir = 'D:\Travaille\Data\signature_square\signature_no'

k=0
nombre_limit = 1
min_score = 1000
max_score = -1
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if k < nombre_limit :
            #print(k)
            img = cv2.imread(os.path.join(subdir, file),0)
            height,width = height_and_width(img)
            dimention_mesure = dimention_mesure_of(height,width)
            L = 30
            #angle = 80
            
            signature_present = False
            maxfirstmax = 0
            for angle in range(0,180,5):
                firstmax,secondmax = filtered_image(img,dimention_mesure,L,angle)
                oblique_firstmax = (angle_weight(angle)**(1/2)) * firstmax #(angle_weight(angle)**(1/2)) *
                if oblique_firstmax > 0.07 :
                    signature_present = True
                    ecart = oblique_firstmax - 0.04
                    #print(ecart)
                    #break;
                
                if oblique_firstmax > maxfirstmax :
                    maxfirstmax = oblique_firstmax
            if signature_present:
                print('signature in : ',file , ' with score : ' , maxfirstmax)
                print()
            else :
                print('No signature in : ',file , ' with score : ' , maxfirstmax)
                print()
            if maxfirstmax < min_score :
                min_score  = maxfirstmax
            if maxfirstmax > max_score:
                max_score  = maxfirstmax
            k=k+1

print('Max score is : ' , max_score)
#print('Min score is : ' , min_score)

('Max score is : ', -1)
